# TRAINING I3W


# A) Create Envorinment, Vehicles etc

### General Parameter

In [1]:
# Define horizon as a variable to ensure consistent use across notebook (length of one rollout)
HORIZON=500                                 #103 max Horizon, wenn es vor verlassen abbrechen soll!, default war 500

# name of the experiment
experiment_name = "IntersectionExample"

# scenario class
import flow.scenarios as scenarios
print("Available scenarios:")
print(scenarios.__all__)
scenario_name = "IntersectionTWScenario"

# environment class
import flow.multiagent_envs as flowenvs
print("\nAvailable environments:")
print(flowenvs.__all__)
env_name = "MultiAgentIntersectionEnv"

Available scenarios:
['Scenario', 'BayBridgeScenario', 'BayBridgeTollScenario', 'BottleneckScenario', 'Figure8Scenario', 'SimpleGridScenario', 'HighwayScenario', 'LoopScenario', 'MergeScenario', 'TwoLoopsOneMergingScenario', 'MultiLoopScenario', 'IntersectionScenarioTW']

Available environments:
['MultiEnv', 'MultiAgentAccelEnv', 'MultiWaveAttenuationPOEnv', 'MultiAgentIntersectionEnv', 'MultiAgentTeamSpiritIntersectionEnv', 'MultiAgentIntersectionEnv_baseline_1', 'MultiAgentIntersectionEnv_baseline_2', 'MultiAgentIntersectionEnv_baseline_3']


### Net Parameter

In [2]:
from flow.core.params import NetParams
from flow.scenarios.intersection import ADDITIONAL_NET_PARAMS

additionalNetParams = {
            "edge_length": 40,
            "lanes": 1,
            "speed_limit": 30
        }

net_params = NetParams( no_internal_links=False,                  #default: True   !! damit Kreuzungen nicht überspr. werden
                        inflows=None,                             #default: None
                        osm_path=None,                            #default: None
                        netfile=None,                             #default: None
                        additional_params=additionalNetParams     #default: None   !!
                      )

### InitialConfig Parameter

In [3]:
from flow.core.params import InitialConfig

initial_config = InitialConfig( shuffle=True,                            #default: False         !!
                                spacing="custom",                        #default: "uniform"     !!
                                min_gap=10,                              #default: 0
                                perturbation=29.99,                      #default: 0.0            !!        
                                x0=0,                                    #default: 0
                                bunching=0,                              #default: 0
                                lanes_distribution=float("inf"),         #default: float("inf")
                                edges_distribution="all",                #default: "all"
                                additional_params=None )                 #default: None

### SUMO Parameter

In [4]:
from flow.core.params import SumoParams

sumo_params = SumoParams( port = None,                  #default: None
                          sim_step=0.1,                 #default: 0.1
                          emission_path=None,           #default: None
                          lateral_resolution=None,      #default: None
                          no_step_log=True,             #default: True
                          render=False,                 #default: False
                          save_render=False,            #default: False
                          sight_radius=25,              #default: 25
                          show_radius=False,            #default: False
                          pxpm=2,                       #default: 2
                          overtake_right=False,         #default: False    
                          seed=None,                    #default: None
                          restart_instance=False,       #default: False
                          print_warnings=True,          #default: True
                          teleport_time=-1,             #default: -1
                          num_clients=1,                #default: 1
                          sumo_binary=None )            #default: None

### Environment Parameter

In [5]:
from flow.core.params import EnvParams

additionalEnvParams = {
        # maximum acceleration of autonomous vehicles
        "max_accel": 3,
        # maximum deceleration of autonomous vehicles
        "max_decel": 3,
        "target_velocity": 30
    }

env_params = EnvParams( additional_params=additionalEnvParams, #default: None    !!
                        horizon=HORIZON,                       #default: 500     !!
                        warmup_steps=0,                        #default: 0       
                        sims_per_step=1,                       #default: 1
                        evaluate=False )                       #default: False

### Vehicles Parameter

In [6]:
from flow.core.params import VehicleParams

# import vehicles dynamics models
#from flow.controllers import SumoCarFollowingController
from flow.controllers import ContinuousRouter
#from flow.controllers.lane_change_controllers import SumoLaneChangeController
from flow.controllers.lane_change_controllers import StaticLaneChanger
from flow.controllers import RLController
from flow.core.params import SumoLaneChangeParams
from flow.core.params import SumoCarFollowingParams
from random import *

vehicles = VehicleParams()

#### Add RL-Agent controlled vehicles 

In [7]:
# car following parameters, default: None
cf_parameter = SumoCarFollowingParams(
                speed_mode="aggressive")
# lane change parameters, default: None
lc_parameter =  None

vehicles.add( # name of the vehicle
                veh_id = "rl",
              # acceleration controller, default: (SumoCarFollowingController, {})
                acceleration_controller=(RLController, {}),
              # lane_change_controller, default: (SumoLaneChangeController, {})
                lane_change_controller=(StaticLaneChanger,{}),
              # routing controller, default: None
                routing_controller=(ContinuousRouter, {}),
              # initial speed, default: 0
                initial_speed=0,
              # number of vehicles, default: 1 
                num_vehicles=2,
                
                car_following_params=cf_parameter
              # speed mode, default: "right_of_way"
                #speed_mode="aggressive",
              # lane change mode, default: "no_lat_collide"
                #lane_change_mode="aggressive", 
              # car following parameter, default: None
                #sumo_car_following_params=cf_parameter,
              # lane change parameter, default: None
                #sumo_lc_params=lc_parameter
)

### Flow Parameter

In [8]:
# Creating flow_params. Make sure the dictionary keys are as specified. 
flow_params = dict( # name of the experiment
                      exp_tag=experiment_name,
                    # name of the flow environment the experiment is running on
                      env_name=env_name,
                    # name of the scenario class the experiment uses
                      scenario=scenario_name,
                    # simulator that is used by the experiment
                      simulator='traci',
                    # sumo-related parameters (see flow.core.params.SumoParams)
                      sim=sumo_params,
                    # environment related parameters (see flow.core.params.EnvParams)
                      env=env_params,
                    # network-related parameters (see flow.core.params.NetParams and
                    # the scenario's documentation or ADDITIONAL_NET_PARAMS component)
                      net=net_params,
                    # vehicles to be placed in the network at the start of a rollout 
                    # (see flow.core.vehicles.Vehicles)
                      veh=vehicles,
                   # (optional) parameters affecting the positioning of vehicles upon 
                   # initialization/reset (see flow.core.params.InitialConfig)
                      initial=initial_config
                )

# B) Training

In [9]:
import json

import ray
try:
    from ray.rllib.agents.agent import get_agent_class
except ImportError:
    from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder

from ray import tune
from ray.rllib.agents.ppo.ppo_policy_graph import PPOPolicyGraph

In [10]:
# number of parallel workers
N_CPUS = 2
# number of rollouts per training iteration
N_ROLLOUTS = 20

ray.init(redirect_output=True, num_cpus=N_CPUS+1)

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-03-20_18-43-52_5179/logs.
Waiting for redis server at 127.0.0.1:58595 to respond...
Waiting for redis server at 127.0.0.1:27052 to respond...
Starting the Plasma object store with 6.554658406 GB memory using /dev/shm.

View the web UI at http://localhost:8889/notebooks/ray_ui.ipynb?token=2ddcf65e490eb487441c0b0eb3c5c1d690941bb451a99eb2



{'node_ip_address': '192.168.2.102',
 'object_store_addresses': ['/tmp/ray/session_2019-03-20_18-43-52_5179/sockets/plasma_store'],
 'raylet_socket_names': ['/tmp/ray/session_2019-03-20_18-43-52_5179/sockets/raylet'],
 'redis_address': '192.168.2.102:58595',
 'webui_url': 'http://localhost:8889/notebooks/ray_ui.ipynb?token=2ddcf65e490eb487441c0b0eb3c5c1d690941bb451a99eb2'}

In [11]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate default 0.999
config["model"].update({"fcnet_hiddens": [100, 50,25]})  # size of hidden layers in network default 64 32
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
#config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
#config["sample_batch_size"] = config["train_batch_size"]/config["num_workers"] # 200 default, trotzdem zu hoch?
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)

In [12]:
# multi agent policy mapping
test_env = create_env()
obs_space = test_env.observation_space
act_space = test_env.action_space

def gen_policy():
    return (PPOPolicyGraph, obs_space, act_space, {})

# Setup PG with an ensemble of `num_policies` different policy graphs
policy_graphs = {'rl_0': gen_policy(), 'rl_1': gen_policy()}
    
def policy_mapping_fn(agent_id):
    return agent_id

config.update({
        'multiagent': {
            'policy_graphs': policy_graphs,
            'policy_mapping_fn': tune.function(policy_mapping_fn)
        }
    })

 Starting SUMO on port 55263


23.147038256975808
6.886284052593682


In [ ]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,  # RL algorithm to run
        "env": gym_name,  # environment name generated earlier
        "config": {  # configuration params (must match "run" value)
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "max_failures": 999,
        "stop": {  # stopping conditions
            "training_iteration": 1000,  # number of iterations to stop after
        },
    },
})

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 0/3 CPUs, 0/1 GPUs
Memory usage on this node: 2.7/16.4 GB

Created LogSyncer for /home/thorsten/ray_results/IntersectionExample/PPO_MultiAgentIntersectionEnv-v0_0_2019-03-20_18-43-55ipg4b7cp -> 
== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 2.7/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_18-44-54
  done: false
  episode_len_mean: 481.7
  episode_reward_max: 157.82593584178935
  episode_reward_mean: 55.08088457653795
  episode_reward_min: -146.92512367743936
  episodes_this_iter: 20
  episodes_total: 20
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 5043.743
    load_time_ms: 151.02
    num_steps_sampled: 10000
    num_steps_trained:

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_18-46-37
  done: false
  episode_len_mean: 453.67
  episode_reward_max: 329.05035258097655
  episode_reward_mean: 89.71178834960858
  episode_reward_min: -158.17230433684242
  episodes_this_iter: 23
  episodes_total: 109
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 4164.82
    load_time_ms: 32.191
    num_steps_sampled: 50000
    num_steps_trained: 50000
    rl_0:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4128468036651611
      kl: 0.007679771166294813
      policy_loss: -0.003738576779142022
      total_loss: 123.07553100585938
      vf_explained_var: 0.28636640310287476
      vf_loss: 123.07918548583984
    rl_1:
      cur_kl_coeff: 0.012500000186264515
      cur_lr: 4.999999873689376e-05
      entropy: 1.4362616539001465
      kl: 0.007967169396579266
      policy_loss: -0.004602141212671995
    

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 234 s, 9 iter, 90000 ts, 212 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_18-48-46
  done: false
  episode_len_mean: 253.02
  episode_reward_max: 389.73737098256055
  episode_reward_mean: 185.773258089166
  episode_reward_min: -161.4407383781322
  episodes_this_iter: 43
  episodes_total: 275
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 4077.173
    load_time_ms: 17.367
    num_steps_sampled: 100000
    num_steps_trained: 100000
    rl_0:
      cur_kl_coeff: 0.0003906250058207661
      cur_lr: 4.999999873689376e-05
      entropy: 1.3957222700119019
      kl: 0.006072198040783405
      policy_loss: -0.001429747324436903
      total_lo

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 364 s, 14 iter, 140000 ts, 270 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_18-50-56
  done: false
  episode_len_mean: 195.04
  episode_reward_max: 318.46425023204347
  episode_reward_mean: 286.4556144171527
  episode_reward_min: -141.1893024718516
  episodes_this_iter: 54
  episodes_total: 522
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 3981.342
    load_time_ms: 2.681
    num_steps_sampled: 150000
    num_steps_trained: 150000
    rl_0:
      cur_kl_coeff: 9.765625145519152e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.3504679203033447
      kl: 0.00482891546562314
      policy_loss: -0.0024597151204943657
      total_

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 3.6/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 494 s, 19 iter, 190000 ts, 284 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_18-53-06
  done: false
  episode_len_mean: 170.39
  episode_reward_max: 316.67231133066366
  episode_reward_mean: 283.61520014457415
  episode_reward_min: -162.2134245326086
  episodes_this_iter: 61
  episodes_total: 808
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 3973.565
    load_time_ms: 2.631
    num_steps_sampled: 200000
    num_steps_trained: 200000
    rl_0:
      cur_kl_coeff: 6.10351571594947e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.346618890762329
      kl: 0.0012807153398171067
      policy_loss: -0.0005645955679938197
      total

...

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 4149 s, 149 iter, 1490000 ts, 325 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_19-54-28
  done: false
  episode_len_mean: 135.2
  episode_reward_max: 361.2902715503085
  episode_reward_mean: 323.06011220079296
  episode_reward_min: 175.4995890002329
  episodes_this_iter: 73
  episodes_total: 10711
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 5290.545
    load_time_ms: 2.948
    num_steps_sampled: 1500000
    num_steps_trained: 1500000
    rl_0:
      cur_kl_coeff: 9.545901775709353e-07
      cur_lr: 4.999999873689376e-05
      entropy: 1.389479398727417
      kl: 1100.154296875
      policy_loss: 0.3395327925682068
      total_loss:

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.1/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 4421 s, 154 iter, 1540000 ts, 325 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_19-58-57
  done: false
  episode_len_mean: 136.05
  episode_reward_max: 365.39965761360526
  episode_reward_mean: 325.3923051364666
  episode_reward_min: 292.1026151221282
  episodes_this_iter: 73
  episodes_total: 11078
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 6689.416
    load_time_ms: 3.586
    num_steps_sampled: 1550000
    num_steps_trained: 1550000
    rl_0:
      cur_kl_coeff: 7.2489151534682605e-06
      cur_lr: 4.999999873689376e-05
      entropy: 1.4288119077682495
      kl: 149.0294647216797
      policy_loss: 0.36122432351112366
      tota

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.2/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 4705 s, 159 iter, 1590000 ts, 325 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_20-03-44
  done: false
  episode_len_mean: 139.72
  episode_reward_max: 383.09491621233127
  episode_reward_mean: 326.4128368254589
  episode_reward_min: 130.87510551995848
  episodes_this_iter: 72
  episodes_total: 11441
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 8502.193
    load_time_ms: 3.948
    num_steps_sampled: 1600000
    num_steps_trained: 1600000
    rl_0:
      cur_kl_coeff: 5.504645378096029e-05
      cur_lr: 4.999999873689376e-05
      entropy: 1.571427822113037
      kl: 60.51083755493164
      policy_loss: 0.30252689123153687
      total

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 6.4/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 4954 s, 164 iter, 1640000 ts, 324 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_20-07-56
  done: false
  episode_len_mean: 138.82
  episode_reward_max: 367.7990810222922
  episode_reward_mean: 323.2182565767242
  episode_reward_min: 292.6989462584217
  episodes_this_iter: 72
  episodes_total: 11799
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 8472.181
    load_time_ms: 4.397
    num_steps_sampled: 1650000
    num_steps_trained: 1650000
    rl_0:
      cur_kl_coeff: 0.00041800900362432003
      cur_lr: 4.999999873689376e-05
      entropy: 1.5089560747146606
      kl: 0.06328733265399933
      policy_loss: 0.020327607169747353
      to

== Status ==
Using FIFO scheduling algorithm.
Resources requested: 3/3 CPUs, 0/1 GPUs
Memory usage on this node: 5.8/16.4 GB
Result logdir: /home/thorsten/ray_results/IntersectionExample
RUNNING trials:
 - PPO_MultiAgentIntersectionEnv-v0_0:	RUNNING [pid=5225], 5110 s, 169 iter, 1690000 ts, 323 rew

Result for PPO_MultiAgentIntersectionEnv-v0_0:
  custom_metrics: {}
  date: 2019-03-20_20-10-11
  done: false
  episode_len_mean: 136.6
  episode_reward_max: 380.4360744755644
  episode_reward_mean: 326.44772270785273
  episode_reward_min: 294.8658171489943
  episodes_this_iter: 73
  episodes_total: 12163
  experiment_id: d4702f73002545e9a3fbdb44c083d921
  hostname: Gandalf
  info:
    grad_time_ms: 5417.15
    load_time_ms: 3.324
    num_steps_sampled: 1700000
    num_steps_trained: 1700000
    rl_0:
      cur_kl_coeff: 0.0021161711774766445
      cur_lr: 4.999999873689376e-05
      entropy: 1.4829151630401611
      kl: 0.058438241481781006
      policy_loss: 0.01931939460337162
      tota